In [1]:
exec(open(f"/Users/yerik/_apple_source/PY/GLOBAL/_1_paths.py",encoding="utf-8").read()) #paths
#direc= curr_direc ;# hi_jupy() 
# curr_direc

ATTN - > TO start coding in this NOTEBOOK & following FNS :::  run ::: hi_jupy()
ATTN at the end RUN > py and cleans py ready to be copied >  runpy()
ATTN - > TO get all GLOBAL, libs and functions & ALL_DATA inputs, stats, temp, tables files info > db_paths()
ATTN - > TO move files from input to curr_direc > move_files(input_folder_path, curr_direc)
ATTN - > TO COPY program > copy_files_and_subdirectories(path_2_copy, dest_direc) --- respects original copied
ATTN - > TO get all GAME PLANS TO EXPORT ALPHA and BETA programs  > game_plan()
define program_folder_name before running runpy()


In [2]:
from pydub import AudioSegment
from pydub.utils import mediainfo

In [3]:
f'{desk_path}/input/ms_podcast/vr.wav'

'/Users/yerik/Desktop/input/ms_podcast/vr.wav'

### importing and converting the audio file to WAV format would be to perform an initial analysis of the audio file.

In [4]:
from pydub import AudioSegment
from pydub.utils import mediainfo
exec(open(f"functions.py",encoding="utf-8").read()) #paths


f'{desk_path}/input/ms_podcast/vr.wav'

'/Users/yerik/Desktop/input/ms_podcast/vr.wav'

# 0 VARIABLES

In [5]:
input_noise_reduction = 1.0 
#(1.0)   Default, no extra reduction),
#1.5)  # More aggressive reduction
# 0.8)  # Less aggressive reduction

# 0 VARIABLES AVOBE
# 1 START PROCESSING BY GETTING A CLEAN WAV BELOW

In [6]:

# Example usage
#convert_to_wav("input.mp3", "output.wav")

convert_to_wav(f'{desk_path}/input/ms_podcast/vr.wav', f'{desk_path}/output/ms_podcast/vr.wav')



File has been converted and saved at /Users/yerik/Desktop/output/ms_podcast/vr.wav


## 2.0  >>  Calculate the length of the audio in seconds and minutes.
## 2.1 >> Determine the number of channels (Mono/Stereo).
## 2.2 >> Retrieve the frame rate and other properties.

In [7]:

length, channels, frame_rate = analyze_audio(f'{desk_path}/output/ms_podcast/vr.wav')  # ^^^ path ^^^
print('\n\n',length, channels, frame_rate)

Audio duration: 165.900952 seconds
Channels: 2
Frame rate: 44100 Hz


 165.900952 2 44100


## 3 >> NOISE REDUTCTION (basic and adjustable)

In [8]:
from scipy.io import wavfile
import numpy as np

In [9]:
# BASIC 
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced.wav'

basic_noise_reduction(input_path, output_path)


Noise-reduced audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_noise_reduced.wav


In [12]:
# ADJUSTABLE 
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced.wav'

adjustable_noise_reduction(input_path, output_path, threshold_factor=input_noise_reduction)  # Less aggressive reduction
print(f'Your Choice was {input_noise_reduction}, \n\n')

Noise-reduced audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_noise_reduced.wav
Your Choice was 1.0, 




# STEM extraction 
!python -m demucs.separate -d cpu "/Users/yerik/Desktop/output/ms_podcast/tostem.wav" -o "/Users/yerik/Desktop/output/ms_podcast/STEMS"


In [ ]:
#copy STEM VOCAL

Explanation
butter_highpass: Creates a high-pass filter using Butterworth design parameters.
Input: cutoff frequency, sampling rate, order of the filter.
Output: Filter coefficients a and b.
highpass_filter: Applies the high-pass filter to audio data.
Input: audio data, cutoff frequency, sampling rate, order of the filter, and threshold factor for aggressive filtering.
Output: Filtered audio data.
adjustable_noise_reduction: A wrapper function that allows users to adjust the noise reduction parameters.
Input: input_path for the input .wav file, output_path for the output .wav file, and threshold_factor for more or less aggressive noise reduction.
Output: Writes the filtered data into a .wav file.

# Hight PASS

In [39]:
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
import numpy as np

def adjustable_highpass_filter(input_path, output_path, cutoff_frequency, filter_order, threshold_factor):

    def butter_highpass(cutoff, fs, order=5):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    def highpass_filter(data, cutoff, fs, order=5, threshold_factor=1):
        b, a = butter_highpass(cutoff, fs, order=order)
        y = filtfilt(b, a, data)
        y = np.where(np.abs(y) > threshold_factor * np.max(np.abs(y)), y, 0)
        return y.astype(np.int16)

    # Read the input audio file
    fs, data = wavfile.read(input_path)  # ^^^ path ^^^
    
    # Check if the audio is stereo
    if len(data.shape) == 2:
        channel1 = data[:, 0]
        channel2 = data[:, 1]
        
        # Apply high-pass filter to each channel
        try:
            filtered_channel1 = highpass_filter(channel1, cutoff_frequency, fs, filter_order, threshold_factor)
            filtered_channel2 = highpass_filter(channel2, cutoff_frequency, fs, filter_order, threshold_factor)
        except ValueError as e:
            print(f"Error: {e}")
            return

        # Merge channels back into a stereo audio file
        filtered_stereo = np.column_stack((filtered_channel1, filtered_channel2))

        # Write the filtered audio to a new file
        wavfile.write(output_path, fs, filtered_stereo)  # ^^^ path ^^^

    else:
        print("The audio file is not in stereo format.")


# Here, replace desk_path, input_noise_reduction, filter_order, and cutoff_frequency with your actual values.
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_now.wav'
input_noise_reduction = 0.05
filter_order = 6
cutoff_frequency = 100.0

adjustable_highpass_filter(input_path, output_path, cutoff_frequency, filter_order, input_noise_reduction)


In [13]:
# from scipy.io import wavfile
# import numpy as np
# import scipy.fftpack

# def advanced_noise_reduction(input_path, output_path):
#     rate, data = wavfile.read(input_path)
#     w = np.fft.fft(data)
#     freqs = np.fft.fftfreq(len(w))
    
#     # Assuming that noise is concentrated at lower frequencies, we apply a high-pass filter
#     # Change these values based on your specific needs
#     min_freq = 500  # in Hz
#     max_freq = 3000  # in Hz
    
#     # Zero out components that are not within the desired frequency range
#     w[(freqs < min_freq)] = 0
#     w[(freqs > max_freq)] = 0
    
#     # Inverse FFT to get the audio back
#     filtered_data = np.fft.ifft(w).real.astype(np.int16)
    
#     # Save the noise-reduced audio
#     wavfile.write(output_path, rate, filtered_data)
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_2.wav'
# advanced_noise_reduction(input_path, output_path)


In [14]:
# from scipy.io import wavfile
# import numpy as np
# import scipy.fftpack

# def advanced_noise_reduction(input_path, output_path):
#     rate, data = wavfile.read(input_path)
#     w = np.fft.fft(data)
#     freqs = np.fft.fftfreq(len(w))
    
#     # Apply a less aggressive high-pass filter
#     min_freq = 100  # in Hz
    
#     # Zero out components that are not within the desired frequency range
#     w[(freqs < min_freq)] = 0
    
#     # Inverse FFT to get the audio back
#     filtered_data = np.fft.ifft(w).real.astype(np.int16)
    
#     # Save the noise-reduced audio
#     wavfile.write(output_path, rate, filtered_data)
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_2.wav'
# advanced_noise_reduction(input_path, output_path)


# high pass

In [16]:
# import librosa
# import scipy.signal
# from scipy.io.wavfile import write

# def simple_highpass_filter(input_path, output_path):
#     # Load audio file
#     y, sr = librosa.load(input_path, sr=None)
    
#     # Create a high-pass filter
#     sos = scipy.signal.butter(10, 100, 'hp', fs=sr, output='sos')
    
#     # Apply filter
#     filtered_signal = scipy.signal.sosfilt(sos, y)
    
#     # Save the noise-reduced audio
#     write(output_path, sr, filtered_signal.astype(np.int16))
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_3.wav'
# simple_highpass_filter(input_path, output_path)


Noise-reduced audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_noise_reduced_3.wav


In [17]:
import librosa
import numpy as np
from scipy.io.wavfile import write

def minimal_processing(input_path, output_path):
    # Load audio file
    y, sr = librosa.load(input_path, sr=None)
    
    # Remove silence
    yt, _ = librosa.effects.trim(y)
    
    # Normalize audio
    yt = librosa.util.normalize(yt)
    
    # Save the audio
    write(output_path, sr, (yt * 32767).astype(np.int16))
    print(f"Processed audio saved at {output_path}")  # ^^^ path ^^^

# Example desk_path for demonstration purposes; replace it with your actual desk_path
#desk_path = "/your/desk/path"

# Example usage
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_minimal_processed.wav'
minimal_processing(input_path, output_path)


Processed audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_minimal_processed.wav


In [ ]:
import librosa
import numpy as np
from scipy.io.wavfile import write

def spectral_subtraction(input_path, output_path, alpha=1.0):
    # Load audio file
    y, sr = librosa.load(input_path, sr=None)
    
    # Compute the short-time Fourier transform (STFT)
    D = librosa.stft(y)
    
    # Estimate the noise
    noise_est = np.median(np.abs(D), axis=1)
    
    # Perform spectral subtraction
    D_denoised = D - alpha * noise_est[:, None]
    
    # Ensure no negative values
    D_denoised = np.maximum(D_denoised, 0.0)
    
    # Inverse STFT to convert back to time domain
    y_denoised = librosa.istft(D_denoised)
    
    # Save the audio
    write(output_path, sr, (y_denoised * 32767).astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# Example desk_path for demonstration purposes; replace it with your actual desk_path
desk_path = "/your/desk/path"

# Example usage
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced.wav'
spectral_subtraction(input_path, output_path, alpha=2.0)


# FUNCTIONS

In [10]:
%%writefile functions.py

def convert_to_wav(input_path, output_path):
    # Supported audio formats by pydub
    SUPPORTED_FORMATS = ["mp3", "flv", "ogg", "mp4", "wav", "aif", "wma", "aac", "m4a"]

    # Extract the file extension from input_path
    file_extension = input_path.split('.')[-1].lower()

    # Check if the file format is supported
    if file_extension not in SUPPORTED_FORMATS:
        print(f"Unsupported file format: {file_extension}")
        return

    # Read the audio file
    audio = AudioSegment.from_file(input_path, format=file_extension)

    # Convert to WAV
    audio.export(output_path, format="wav")
    print(f"File has been converted and saved at {output_path}")  # ^^^ path ^^^


# ------------------------------------------------------------------- Function to analyze audio
def analyze_audio(audio_path):
    #from pydub.utils import mediainfo
    audio_info = mediainfo(audio_path)
    length_in_seconds = float(audio_info["duration"])
    channels = int(audio_info["channels"])
    frame_rate = int(audio_info["sample_rate"])

    print(f"Audio duration: {length_in_seconds} seconds")
    print(f"Channels: {channels}")
    print(f"Frame rate: {frame_rate} Hz")
    
    return length_in_seconds, channels, frame_rate


## ------------------------------------------------------------------- Noise Reduction 

def basic_noise_reduction(input_path, output_path):
    # Read the input audio file
    rate, data = wavfile.read(input_path)
    
    # Take a sample of the noise (adjust as needed)
    noise_sample = data[:5000]
    
    # Generate noise profile
    noise_profile = np.mean(np.abs(noise_sample))
    
    # Perform noise reduction: Zero out anything below the profile
    reduced_noise_data = np.where(np.abs(data) < noise_profile, 0, data)
    
    # Save the noise-reduced audio
    wavfile.write(output_path, rate, reduced_noise_data.astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# ## ------------------------------------------------------------------- ADJUSTABLE Noise Reduction 


def adjustable_noise_reduction(input_path, output_path, threshold_factor=1.0):
    # Read the input audio file
    rate, data = wavfile.read(input_path)
    
    # Take a sample of the noise (adjust the range as needed)
    noise_sample = data[:5000]
    
    # Generate noise profile
    noise_profile = np.mean(np.abs(noise_sample))
    
    # Calculate the adjusted threshold
    adjusted_threshold = noise_profile * threshold_factor
    
    # Perform noise reduction: Zero out anything below the adjusted threshold
    reduced_noise_data = np.where(np.abs(data) < adjusted_threshold, 0, data)
    
    # Save the noise-reduced audio
    wavfile.write(output_path, rate, reduced_noise_data.astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

Overwriting functions.py
